In [1]:
#%% Necessary packages
# 1. Keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import regularizers
from keras import backend as K
K.set_image_data_format('channels_first')
# 2. Others
import tensorflow as tf
import numpy as np

# 3. PyTorch
import torch
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from skimage import io, transform, color
import PIL
from PIL import Image
import pandas as pd
import os

#ignore warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

/home/audy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/audy/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [17]:
class dataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):

        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.mask_dir = self.root_dir.replace('CBIS-DDSM_classification','masks')

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,self.data_frame.iloc[idx]['name'])
        image = Image.open(img_name)

        label = self.data_frame.iloc[idx]['category']

        mask_name = os.path.join(self.mask_dir,self.data_frame.iloc[idx]['name'].replace('.j','_mask.j'))
        mask = io.imread(mask_name)
        mask = np.array([mask,mask,mask]).transpose((1,2,0))
        mask = Image.fromarray(mask)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask) 
      
        return {'image':image,'category':label,'mask':mask, 'name':img_name}
    

def get_dataloader(data_dir, train_csv_path, image_size, img_mean, img_std, batch_size=1):

    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize(image_size),#row to column ratio should be 1.69
            #transforms.RandomHorizontalFlip(0.5),
            transforms.RandomVerticalFlip(0.5),
            transforms.RandomRotation(30),
            transforms.ToTensor(),
            #transforms.Normalize([0.223, 0.231, 0.243], [0.266, 0.270, 0.274])
            transforms.Normalize(img_mean,img_std)
        ]),
        'valid': transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            #transforms.Normalize([0.223, 0.231, 0.243], [0.266, 0.270, 0.274])
            transforms.Normalize(img_mean,img_std)
        ]),
        'test': transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            #transforms.Normalize([0.223, 0.231, 0.243], [0.266, 0.270, 0.274])
            transforms.Normalize(img_mean,img_std)
        ])
    }

    image_datasets = {}
    dataloaders = {}
    dataset_sizes = {}

    for x in ['train', 'valid', 'test']:
        image_datasets[x] = dataset(train_csv_path.replace('train',x),root_dir=data_dir,transform=data_transforms[x])

        if x!= 'test':
            dataloaders[x] = torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,shuffle=True, num_workers=4)
        else:
            dataloaders[x] = torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,shuffle=False, num_workers=4)
        dataset_sizes[x] = len(image_datasets[x])

    device = torch.device("cuda:0")

    return dataloaders,dataset_sizes,image_datasets,device

In [12]:
data_dir = '../Data/CBIS-DDSM_classification_orient/'
train_csv = '../CSV/gain_train.csv'
#image_size = (640,384)
image_size = (320,192)
batch_size = 12
img_mean = [0.223, 0.231, 0.243]
img_std = [0.266, 0.270, 0.274]

dataloaders,dataset_sizes,dataset,device = get_dataloader(data_dir,train_csv,image_size,img_mean,img_std,batch_size)


a = iter(dataloaders['train']).next()

In [ ]:
#%% Define PVS class
class PVS():
    
    # 1. Initialization
    '''
    x_train: training samples
    data_type: Syn1 to Syn 6
    '''
    def __init__(self, x_train, data_type, lamda):
        
        self.data_dir = '../Data/CBIS-DDSM_classification_orient/'
        self.train_csv = '../CSV/gain_train.csv'
        self.input_shape = (320,192)
        self.batch_size = 12
        self.img_mean = [0.223, 0.231, 0.243]
        self.img_std = [0.266, 0.270, 0.274]
        self.alpha = 

        # Use Adam optimizer with learning rate = 0.0001
        optimizer = Adam(0.0001)
        
        # Build and compile the predictor
        self.predictor = self.build_predictor_discriminator('predictor')
        # Use categorical cross entropy as the loss
        self.predictor.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

        # Build and compile the predictor
        self.discriminator = self.build_predictor_discriminator('discriminator')
        # Use categorical cross entropy as the loss
        self.discriminator.compile(loss=self.my_nce_loss(), optimizer=optimizer, metrics=['acc'])

        # Build the generator (actor)
        self.selector = self.build_selector()
        # Use custom loss (my loss)
        self.selector.compile(loss=self.my_loss, optimizer=optimizer)

    def my_nce_loss(y_true, y_pred):
        return -K.categorical_crossentropy(y_true, y_pred)
        
    #%% Custom loss definition
    def my_loss(self, y_true, y_pred):
        
        # dimension of the features
        d = y_true.shape[1]//3        
        
        # Put all three in y_true 
        # 1. predictor probability
        p_prob = y_true[:,:d]
        # 2. discriminator output
        d_prob = y_true[:,d:2*d]
        # 3. ground truth
        y_final = y_true[:,2*d:]
        
        ce_pred = tf.reduce_sum(K.categorical_crossentropy(y_final, p_prob, axis=1))
        ce_dis = tf.reduce_sum(K.categorical_crossentropy(y_final, d_prob, axis=1))
        l2_norm = tf.reduce_sum(tf.norm(y_pred, ord='euclidean', axis=[1,2,3]))

        loss = tf.reduce_mean(self.alpha*ce_pred - ce_dis + self.beta*l2_norm)

        return loss

    #%% Selector
    def build_selector():

        inputs = Input((256,256,1))
        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
        drop4 = Dropout(0.5)(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

        conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
        conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
        drop5 = Dropout(0.5)(conv5)

        up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
        #merge6 = merge([drop4,up6], mode = 'concat', concat_axis = 3)
        merge6 = Concatenate()([drop4,up6])#, axis=-1)
        conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
        conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

        up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
        #merge7 = merge([conv3,up7], mode = 'concat', concat_axis = 3)
        merge7 = Concatenate()([conv3,up7])#, axis=-1)
        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

        up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
        #merge8 = merge([conv2,up8], mode = 'concat', concat_axis = 3)
        merge8 = Concatenate()([conv2,up8])#, axis=-1)
        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

        up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
        #merge9 = merge([conv1,up9], mode = 'concat', concat_axis = 3)
        merge9 = Concatenate()([conv1,up9])#, axis=-1)
        conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
        conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

        model = Model(inputs = inputs, outputs = conv10)

        model.summary()

        return model


    #%% Discriminator (Critic)
    #We will share weights between the predictor and the discriminator. 
    #mode = predictor => select = select
    #mode = discriminator => select = 1 - select
    
    def build_predictor_discriminator(self,mode):
        
        # create the base pre-trained model
        base_model = InceptionV3(weights='imagenet', include_top=False)

        # add a global spatial average pooling layer
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        # let's add a fully-connected layer
        x = Dense(1024, activation='relu')(x)
        # and a logistic layer -- we have 2 classes
        predictions = Dense(2, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=predictions)

        model.summary()
        
        # There are two inputs to be used in the discriminator
        # 1. Features
        feature = Input(shape=(self.input_shape,), dtype='float32')
        # 2. Selected Features
        select = Input(shape=(self.input_shape,), dtype='float32')         
        
        # Element-wise multiplication
        if mode == 'discriminator':
            select = tf.subtract(tf.ones(tf.TensorShape(select)),select)
            
        model_input = Multiply()([feature, select])
        prob = model(model_input)

        return Model([feature, select], prob)

    #%% Sampling the features based on the output of the generator
    def Sample_M(self, gen_prob):
        
        # Shape of the selection probability
        n = gen_prob.shape[0]
        h = gen_prob.shape[1]
        w = gen_prob.shape[2]
            
        # Sampling
        samples = np.random.binomial(1, gen_prob, (n,d,w))
        
        return samples

    #%% Training procedure
    def train(self, x_train, y_train):
        
        data_dir = '../Data/CBIS-DDSM_classification_orient/'
        train_csv = '../CSV/gain_train.csv'
        #image_size = (640,384)
        image_size = (320,192)
        batch_size = 12
        img_mean = [0.223, 0.231, 0.243]
        img_std = [0.266, 0.270, 0.274]

        dataloaders,dataset_sizes,dataset,device = get_dataloader(data_dir,train_csv,image_size,img_mean,img_std,batch_size)


        # For each epoch (actually iterations)
        for epoch in range(self.epochs):

            
            #%% Train Discriminator
            # Select a random batch of samples
            idx = np.random.randint(0, x_train.shape[0], self.batch_size)
            x_batch = x_train[idx,:]
            y_batch = y_train[idx,:]

            # Generate a batch of probabilities of feature selection
            sel_prob = self.generator.predict(x_batch)
            
            # Sampling the features based on the generated probability
            #sel_prob = self.Sample_M(gen_prob)     
            
            # Compute the prediction of the critic based on the sampled features (used for generator training)
            pred_prob = self.predictor.predict([x_batch, sel_prob])
            
            dis_prob = self.discriminator.predict([x_batch, sel_prob])

            # Train the predictor
            p_loss = self.predictor.train_on_batch([x_batch, sel_prob], y_batch)
            
            # Train the discriminator
            d_loss = self.discriminator.train_on_batch([x_batch, sel_prob], y_batch)

            #%% Train selector
            # Use three things as the y_true: sel_prob, dis_prob, and ground truth (y_batch)
            y_batch_final = np.concatenate( (np.asarray(p_prob), np.asarray(dis_prob), y_batch), axis = 1 )
            #y_batch_final = [np.asarray(pred_prob),np.asarray(dis_prob),y_batch]
            
            # Train the generator
            g_loss = self.selector.train_on_batch(x_batch, y_batch_final)

            #%% Plot the progress
            dialog = 'Epoch: ' + str(epoch) + ', p_loss (CE): ' + str(np.round(p_loss[0],4)) + ', p_loss (Acc): ' + str(p_loss[1]) + ', g_loss: ' + str(np.round(g_loss,4))
 
            if epoch % 100 == 0:              
                print(dialog)
    
    #%% Selected Features        
    def output(self, x_train):
        
        gen_prob = self.selector.predict(x_train)
        
        return np.asarray(gen_prob)


#%% Main Function
if __name__ == '__main__':
        
    # Data generation function import
    from Data_Generation import generate_data
    
    #%% Parameters
    # Synthetic data type    
    idx = 5
    data_sets = ['Syn1','Syn2','Syn3','Syn4','Syn5','Syn6']
    data_type = data_sets[idx]
    
    # No need to provide the number of relevant features at all!

    # Data output can be either binary (Y) or Probability (Prob)
    data_out_sets = ['Y','Prob']
    data_out = data_out_sets[0]
    
    # Number of Training and Testing samples
    train_N = 10000
    test_N = 10000
    
    # Seeds (different seeds for training and testing)
    train_seed = 0
    test_seed = 1
        
    #%% Data Generation (Train/Test)
    def create_data(data_type, data_out): 
        
        x_train, y_train, g_train = generate_data(n = train_N, data_type = data_type, seed = train_seed, out = data_out)  
        x_test,  y_test,  g_test  = generate_data(n = test_N,  data_type = data_type, seed = test_seed,  out = data_out)  
    
        return x_train, y_train, g_train, x_test, y_test, g_test
    
    x_train, y_train, g_train, x_test, y_test, g_test = create_data(data_type, data_out)

    #%% Hyperparameter
    lamda = 3

    # 1. PVS Class call
    PVS_Alg = PVS(x_train, data_type, lamda)
        
    # 2. Algorithm training
    PVS_Alg.train(x_train, y_train)    
    
    # 3. Get the selection probability on the testing set
    Sel_Prob_Test = PVS_Alg.output(x_test)
    
    # 4. Selected features
    score = 1.*(Sel_Prob_Test > 0.5)
    
    #%% Performance Metrics
    def performance_metric(score, g_truth):

        n = len(score)
        Temp_TPR = np.zeros([n,])
        Temp_FDR = np.zeros([n,])
        
        for i in range(n):
    
            # TPR    
            TPR_Nom = np.sum(score[i,:] * g_truth[i,:])
            TPR_Den = np.sum(g_truth[i,:])
            Temp_TPR[i] = 100 * float(TPR_Nom)/float(TPR_Den+1e-8)
        
            # FDR
            FDR_Nom = np.sum(score[i,:] * (1-g_truth[i,:]))
            FDR_Den = np.sum(score[i,:])
            Temp_FDR[i] = 100 * float(FDR_Nom)/float(FDR_Den+1e-8)
    
        return np.mean(Temp_TPR), np.mean(Temp_FDR), np.std(Temp_TPR), np.std(Temp_FDR)
    
    #%% Output
        
    TPR_mean, FDR_mean, TPR_std, FDR_std = performance_metric(score, g_test)
        
    print('TPR mean: ' + str(np.round(TPR_mean,1)) + '\%, ' + 'TPR std: ' + str(np.round(TPR_std,1)) + '\%, '  )
    print('FDR mean: ' + str(np.round(FDR_mean,1)) + '\%, ' + 'FDR std: ' + str(np.round(FDR_std,1)) + '\%, '  )